In [1]:
import pandas as pd
import torch

In [2]:
df = pd.read_csv('voice.csv')
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [3]:
x = df.values[:, :-1]
x = torch.tensor(x.astype('float32'))
y = df.values[:, -1]
print('y', y)
y[y == 'male'] = 0
y[y == 'female'] = 1
print('y', y)
y = torch.tensor(y.astype('float32'))

y ['male' 'male' 'male' ... 'female' 'female' 'female']
y [0 0 0 ... 1 1 1]


In [6]:
from torch.utils.data import random_split, DataLoader, Dataset

In [7]:
class TensorDataset(Dataset):
    def __init__(self, *tensors, mu, std):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors), "Size mismatch between tensors"
        self.tensors = tensors
        self.mu = mu
        self.std = std

    def __getitem__(self, index):
        x = self.tensors[0][index]
        x = (x - self.mu) / self.std
        y = self.tensors[1][index]
        return (x, y)

    def __len__(self):
        return self.tensors[0].size(0)

In [8]:
train_size = round(len(x) * 0.7)
test_size = len(x) - train_size
train_idx, test_idx = random_split(range(len(x)), [train_size, test_size])
mu = x[train_idx].mean(dim=0)
std = x[train_idx].std(dim=0)

In [9]:
dataset_train = TensorDataset(x[train_idx], y[train_idx], mu=mu, std=std)
dataset_test = TensorDataset(x[test_idx], y[test_idx], mu=mu, std=std)

In [8]:
print('train', len(dataset_train))
print('test', len(dataset_test))

train 2218
test 950


In [58]:
sum(test_y) / len(test_y)

0.5010526315789474

In [57]:
sum(train_y) / len(train_y)

0.4995491433724076

In [43]:
train_x = dataset_train.tensors[0].numpy()
train_y = dataset_train.tensors[1].numpy()
test_x = dataset_test.tensors[0].numpy()
test_y = dataset_test.tensors[1].numpy()

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

In [51]:
lg_model = LogisticRegression(solver='lbfgs', max_iter=1000)
lg_model.fit(train_x, train_y)

train_pred = lg_model.predict(train_x)
test_pred = lg_model.predict(test_x)
print('Train ACC: {}'.format(accuracy_score(train_y, train_pred)))
print('Test ACC: {}'.format(accuracy_score(test_y, test_pred)))

Train ACC: 0.9075743913435528
Test ACC: 0.9157894736842105


In [53]:
dt_model = DecisionTreeClassifier()
dt_model.fit(train_x, train_y)

train_pred = dt_model.predict(train_x)
test_pred = dt_model.predict(test_x)
print('Train ACC: {}'.format(accuracy_score(train_y, train_pred)))
print('Test ACC: {}'.format(accuracy_score(test_y, test_pred)))

Train ACC: 1.0
Test ACC: 0.9568421052631579


In [59]:
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(train_x, train_y)

train_pred = rf_model.predict(train_x)
test_pred = rf_model.predict(test_x)
print('Train ACC: {}'.format(accuracy_score(train_y, train_pred)))
print('Test ACC: {}'.format(accuracy_score(test_y, test_pred)))

Train ACC: 1.0
Test ACC: 0.9789473684210527
